In [1]:
pip install tensorflow pandas numpy pillow scikit-learn

Note: you may need to restart the kernel to use updated packages.


In [2]:
import os
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing import image
from PIL import Image


In [1]:
import tensorflow as tf
print("TensorFlow version:", tf.__version__)
print("Num GPUs Available: ", len(tf.config.list_physical_devices('GPU')))

TensorFlow version: 2.16.1
Num GPUs Available:  0


In [ ]:
pip uninstall tensorflow

In [ ]:
pip uninstall tensorflow

In [3]:
# Load the dataset
df = pd.read_csv('C:/Users/DELL/Desktop/backend-nutrition-count/Last_cleaned_dataset.csv')
print(df.head())

   calories  proteins   fat  carbohydrate
0     280.0       9.2  28.4           0.0
1     513.0      23.7  37.0          21.3
2       0.0       0.0   0.2           0.0
3      45.0       1.1   0.4          10.8
4      37.0       4.4   0.5           3.8


In [4]:
# Define image directory
image_dir = 'C:/Users/DELL/Desktop/backend-nutrition-count/images'

# Check if images directory exists
if not os.path.exists(image_dir):
    raise FileNotFoundError(f"Image directory '{image_dir}' does not exist.")


In [5]:
# Prepare image paths and labels
image_paths = []
labels = []

for index, row in df.iterrows():
    img_path = os.path.join(image_dir, f"{index}.jpg")  # Assuming images are named as 0.jpg, 1.jpg, etc.
    if os.path.exists(img_path):
        image_paths.append(img_path)
        labels.append([row['calories'], row['proteins'], row['fat'], row['carbohydrate']])
    else:
        print(f"Image {img_path} not found, skipping.")




In [6]:
# Convert labels to numpy array
labels = np.array(labels)

# Split the dataset into training and validation sets
train_image_paths, val_image_paths, train_labels, val_labels = train_test_split(image_paths, labels, test_size=0.2, random_state=42)


In [7]:

# Function to load and preprocess images
def load_and_preprocess_image(img_path, target_size=(128, 128)):
    try:
        img = image.load_img(img_path, target_size=target_size)
        img_array = image.img_to_array(img)
        img_array /= 255.0  # Normalize to [0, 1] range
        return img_array
    except Exception as e:
        print(f"Error loading image {img_path}: {e}")
        return None

In [8]:
# Custom generator to yield batches of images and labels
def custom_image_generator(image_paths, labels, batch_size=32, target_size=(128, 128)):
    while True:
        for start in range(0, len(image_paths), batch_size):
            end = min(start + batch_size, len(image_paths))
            batch_image_paths = image_paths[start:end]
            batch_labels = labels[start:end]
            batch_images = []
            for img_path in batch_image_paths:
                img = load_and_preprocess_image(img_path, target_size)
                if img is not None:
                    batch_images.append(img)
            yield np.array(batch_images), np.array(batch_labels)

# Create the data generators
train_generator = custom_image_generator(train_image_paths, train_labels, batch_size=32, target_size=(128, 128))
val_generator = custom_image_generator(val_image_paths, val_labels, batch_size=32, target_size=(128, 128))

In [9]:
### Step 2: Define the CNN Model

# Define the CNN model architecture
model = Sequential([
    Conv2D(32, (3, 3), activation='relu', input_shape=(128, 128, 3)),
    MaxPooling2D((2, 2)),
    Conv2D(64, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Conv2D(128, (3, 3), activation='relu'),
    MaxPooling2D((2, 2)),
    Flatten(),
    Dense(128, activation='relu'),
    Dropout(0.5),
    Dense(4, activation='linear')  # 4 outputs for calories, proteins, fat, carbohydrate
])


C:\Users\DELL\anaconda3\lib\site-packages\keras\src\layers\convolutional\base_conv.py:99: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(


In [10]:
# Compile the model
model.compile(optimizer=Adam(learning_rate=0.001), loss='mean_squared_error', metrics=['mae'])

In [11]:


### Step 3: Train the Model

# Train the model
history = model.fit(
    train_generator,
    validation_data=val_generator,
    epochs=50,  # Adjust the number of epochs as needed
    steps_per_epoch=len(train_image_paths) // 32,
    validation_steps=len(val_image_paths) // 32
)



Epoch 1/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 17s 541ms/step - loss: 13149.7734 - mae: 58.1382 - val_loss: 5596.0273 - val_mae: 40.0797
Epoch 2/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 13s 477ms/step - loss: 10146.8613 - mae: 55.1531 - val_loss: 4894.8467 - val_mae: 40.0097
Epoch 3/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 12s 430ms/step - loss: 9162.3271 - mae: 53.0624 - val_loss: 4956.1763 - val_mae: 40.7208
Epoch 4/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 11s 409ms/step - loss: 8715.3604 - mae: 49.6962 - val_loss: 5394.3804 - val_mae: 39.1089
Epoch 5/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 12s 442ms/step - loss: 8429.5088 - mae: 49.3908 - val_loss: 4821.3677 - val_mae: 37.4004
Epoch 6/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 12s 425ms/step - loss: 8052.3418 - mae: 47.6257 - val_loss: 4818.0229 - val_mae: 37.5054
Epoch 7/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 12s 431ms/step - loss: 8120.3589 - mae: 46.9010 - val_loss: 4787.2437 - val_mae: 39.9604
Epoch 8/50
28/28 ━━━━━━━━━━━━━━━━━━━━ 12s 435ms/step - loss: 6811.0918 - mae: 44.5259 - val_loss: 4794.4077 -

In [ ]:
# Step 4: Evaluate the Model

# Evaluate the model on the validation set
val_loss, val_mae = model.evaluate(val_generator)
print(f"Validation Loss: {val_loss}")
print(f"Validation MAE: {val_mae}")

# Save the model
#model.save('nutrition_cnn_model.h5')

print("Model training complete and saved as 'nutrition_cnn_model.h5'.")

   2668/Unknown 664s 249ms/step - loss: 6633.0845 - mae: 41.2212

In [29]:
model.save('nutrition_cnn_model.h5')

In [21]:

# Function to load and preprocess a single image
def load_and_preprocess_image(img_path, target_size=(128, 128)):
    try:
        img = image.load_img(img_path, target_size=target_size)
        img_array = image.img_to_array(img)
        img_array /= 255.0 
        return np.expand_dims(img_array, axis=0)  
    except Exception as e:
        print(f"Error loading image {img_path}: {e}")
        return None


img_path = 'C:/Users/DELL/Desktop/backend-nutrition-count/test.jpg'
input_image = load_and_preprocess_image(img_path)

if input_image is not None:
    
   
    prediction = model.predict(input_image)


    predicted_calories, predicted_proteins, predicted_fat, predicted_carbohydrate = prediction[0]

    # Print the predicted values
    print(f"Predicted Calories: {predicted_calories}")
    print(f"Predicted Proteins: {predicted_proteins}")
    print(f"Predicted Fat: {predicted_fat}")
    print(f"Predicted Carbohydrate: {predicted_carbohydrate}")
else:
    print("Failed to load and preprocess the image.")


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 47ms/step
Predicted Calories: -0.007841017097234726
Predicted Proteins: 0.058247048407793045
Predicted Fat: 0.21332192420959473
Predicted Carbohydrate: -0.010748237371444702


In [28]:
import tensorflow as tf

# Check if TensorFlow is built with CUDA (GPU support)
print("Is TensorFlow built with CUDA:", tf.test.is_built_with_cuda())

# List physical devices available
physical_devices = tf.config.list_physical_devices()
print("Physical devices:", physical_devices)

# Check if there is a GPU available
gpu_available = tf.config.list_physical_devices('GPU')
print("GPU available:", gpu_available)


Is TensorFlow built with CUDA: False
Physical devices: [PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU')]
GPU available: []


In [24]:
from tensorflow.keras.models import load_model
def load_and_preprocess_image(img_path, target_size=(128, 128)):
    try:
        img = image.load_img(img_path, target_size=target_size)
        img_array = image.img_to_array(img)
        img_array /= 255.0 
        return np.expand_dims(img_array, axis=0)  
    except Exception as e:
        print(f"Error loading image {img_path}: {e}")
        return None

model = load_model('C:/Users/DELL/Desktop/backend-nutrition-count/nutrition_cnn_model.h5')
img_path = 'C:/Users/DELL/Desktop/backend-nutrition-count/Ojja.jpg'
input_image = load_and_preprocess_image(img_path)
if input_image is not None:
    
   
    prediction = model.predict(input_image)


    predicted_calories, predicted_proteins, predicted_fat, predicted_carbohydrate = prediction[0]

    # Print the predicted values
    print(f"Predicted Calories: {predicted_calories}")
    print(f"Predicted Proteins: {predicted_proteins}")
    print(f"Predicted Fat: {predicted_fat}")
    print(f"Predicted Carbohydrate: {predicted_carbohydrate}")
else:
    print("Failed to load and preprocess the image.")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 231ms/step
Predicted Calories: 151.5972137451172
Predicted Proteins: 9.150940895080566
Predicted Fat: 5.534814834594727
Predicted Carbohydrate: 21.080930709838867
